In [8]:
import numpy as np 
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU,Dense
from tensorflow.keras.optimizers import Adam
import datetime
 

In [9]:
# Step 2: Download stock data (last 10 years till yesterday)

ticker = "AAPL"  # Stock symbol (can be changed)
end_date = datetime.date.today() - datetime.timedelta(days=1)
start_date = end_date - datetime.timedelta(days=365 * 10)

df = yf.download(ticker, start=start_date, end=end_date)

# Display first and last few rows of data
print(df.head())
print(df.tail())


# Step 3: Select multiple features for model input

features = ['Open', 'High', 'Low', 'Close', 'Volume']
data = df[features].values  # Convert selected features to NumPy array

C:\Users\ranji\AppData\Local\Temp\ipykernel_16000\1218677942.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2015-12-22  24.177040  24.287519  24.001173  24.215369  131157600
2015-12-23  24.488180  24.542292  24.170268  24.186051  130629600
2015-12-24  24.357412  24.576118  24.339374  24.576118   54281600
2015-12-28  24.084591  24.280749  23.940291  24.258201  106816800
2015-12-29  24.517492  24.673066  24.093611  24.116158  123724800
Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2025-12-12  278.279999  279.220001  276.820007  277.899994  39532900
2025-12-15  274.109985  280.149994  272.839996  280.149994  50409100
2025-12-16  274.609985  275.500000  271.790009  272.820007  37648600
2025-12-17  271.839996  276.160004  271.640015  275.010010

In [10]:
#step 3:scale Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data =scaler.fit_transform(data)
scaled_data[:5]#print frist 5 rows of scaled data


array([[0.01388484, 0.01262587, 0.01367666, 0.01345313, 0.21627247],
       [0.01377449, 0.01357753, 0.01431987, 0.01462465, 0.21524391],
       [0.01524271, 0.01370388, 0.01496312, 0.01413228, 0.0665169 ],
       [0.01404606, 0.01260058, 0.01344508, 0.01310503, 0.16885623],
       [0.01351141, 0.01406602, 0.01402828, 0.01473502, 0.20179326]])

In [13]:
#step 4 create Dataset function 
def create_dataset(data,tine_step=0):
    x,y=[],[]
    for i in range(len(data)- time_step -1):
        x.append(data[i:(i+ time_step)])
        y.append(data[i + time_step,[0,3]])
    return np.array(x), np.array(y)
time_step=60
x,y =create_dataset(scaled_data,time_step)

#Reshape into 3D :[sampels,time_steps,featurs]
x=x.reshape(x.shape[0],x.shape[1],x.shape[2])

In [15]:
# Step 5: Build GRU Model

model = Sequential()

model.add(
    GRU(
        units=64,
        return_sequences=True,
        input_shape=(time_step, x.shape[2])
    )
)

model.add(GRU(units=64))

model.add(Dense(units=2))  # Output both Open & Close

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

c:\Users\ranji\Downloads\Deep learning\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
# Step 6: Train Model

model.fit(x, y, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0086
Epoch 2/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - loss: 2.2872e-04
Epoch 3/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - loss: 1.8791e-04
Epoch 4/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 1.5904e-04
Epoch 5/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 1.4898e-04
Epoch 6/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 1.3918e-04
Epoch 7/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 1.2516e-04
Epoch 8/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 1.2084e-04
Epoch 9/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 1.1635e-04
Epoch 10/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 1.1946e-04


In [ ]:
#step 7 : predict Today's open & close 
last_60_days = scaled_data[-time_step:] #last 60 rows
#print("last 60 days:",last_60_days)
last_60_days=last_60_days.reshape(1, time_step,x.shape[2])# reshape to 3D and include all features and batch
print("Reshaped last 60 days data:",last_60_days)
predicted_prices= model.predict(last_60_days)

print("scaled predicted prices (open)")